# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [6]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('my_key')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [37]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [ ]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [ ]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [ ]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

In [ ]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [ ]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [ ]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2009?", context_user))

Few Shots for classification.


In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

## Test 1

In [ ]:
context_movie = [
    {'role':'system', 'content':
     """You are an expert film critic who classifies movies into their primary and secondary genres.

     Movie: The Silence of the Lambs
     Primary Genre: Thriller
     Secondary Genre: Crime
     Notable Elements: Psychological, Detective

     Movie: Shaun of the Dead
     Primary Genre: Comedy
     Secondary Genre: Horror
     Notable Elements: Satire, Zombies
     """}
]
print(return_OAIResponse("Classify the movie 'Inception'", context_movie))

## Test 2

In [ ]:
context_recipe = [
    {'role':'system', 'content':
     """You are a master chef who rates recipes based on their complexity.

     Recipe: Boiled Eggs
     Difficulty: ★☆☆☆☆
     Time: 10 minutes
     Skills Needed: Basic timing

     Recipe: Beef Wellington
     Difficulty: ★★★★★
     Time: 180 minutes
     Skills Needed: Advanced pastry work, meat preparation, temperature control
     """}
]
print(return_OAIResponse("Rate this recipe: Homemade Pizza from scratch", context_recipe))

## Test 3

In [ ]:
context_bugs = [
    {'role':'system', 'content':
     """You are a software QA expert who classifies bug reports.

     Bug: App crashes when loading large files
     Severity: CRITICAL
     Impact: User cannot work with essential features
     Priority: Immediate fix required

     Bug: Button color doesn't match design
     Severity: LOW
     Impact: Visual inconsistency only
     Priority: Can be fixed in future release
     """}
]
print(return_OAIResponse("Classify this bug: User passwords are stored in plain text", context_bugs))

## Test 4

In [ ]:
# Surfing Spot Analyzer
context_surf = [
    {'role':'system', 'content':
     """You are an expert surf instructor who analyzes surf spots and provides detailed recommendations.

     Spot: Pipeline, Hawaii
     Wave Conditions: Large, hollow waves, powerful reef break
     Best Swell Direction: NW to N
     Wave Height: 6-12 feet
     Skill Level: Expert Only
     Recommended Board: 6'6" to 7'2" gun or semi-gun
     Best Season: Winter (November to April)

     Spot: Byron Bay, Australia
     Wave Conditions: Long, rolling waves, sand bottom
     Best Swell Direction: E to SE
     Wave Height: 2-6 feet
     Skill Level: Beginner to Intermediate
     Recommended Board: 7'0" to 9'0" longboard or minimal
     Best Season: Summer (December to February)

     Spot: Tres Palmas, Puerto Rico
     Wave Conditions: Long, powerful waves, rocky reef break
     Best Swell Direction: NW to NNE
     Wave Height: 8-25 feet
     Skill Level: Advance to Expert 
     Recommended Board: 6'0" to 9'0" step-up and guns surdboards
     Best Season: Summer (November to April)
     """}
]

# Test different surf spots
print("Analyzing Nazaré, Portugal:")
print(return_OAIResponse("What are the conditions for surfing in Nazaré, Portugal?", context_surf))

print("\nAnalyzing Waikiki, Hawaii:")
print(return_OAIResponse("What are the conditions for surfing in Waikiki Beach?", context_surf))

print("\nAnalyzing Tres Palmas, Puerto Rico:")
print(return_OAIResponse("What are the conditions for surfing in Tres palmas Beach?", context_surf))

print("\nAnalyzing Uluwatu, Bali:")
print(return_OAIResponse("What are the conditions for surfing in Uluwatu, Bali?", context_surf))

print("\nAnalyzing Margara, Puerto Rico:")
print(return_OAIResponse("What are the conditions for surfing in Margara, Puerto Rico?", context_surf))

## Summary Report: Few-Shot Learning Experiments

## Overview
In this notebook, we explored different approaches to few-shot learning using GPT-3.5, testing various formats and domains. We implemented four distinct test cases: movie classification, recipe difficulty rating, bug severity classification, and surf spot analysis.

## Key Findings

### What Worked Well
1. **Consistent Formatting**: The model maintained the specified format across all examples, showing strong ability to follow patterns.
2. **Domain Adaptation**: Successfully adapted to different domains (movies, cooking, tech, surfing) while maintaining accuracy.
3. **Complex Classifications**: Handled multi-dimensional classifications well (e.g., primary/secondary genres, multiple surf conditions).

### What Didn't Work Well
1. **Numerical Precision**: In the surf spot analysis, the model sometimes provided overly specific numbers for wave heights and conditions.
2. **Local Knowledge**: For lesser-known surf spots (e.g., Margara), the model filled in gaps with plausible but potentially inaccurate information.
3. **Consistency**: When testing multiple surf spots, some responses varied in detail level.

## Lessons Learned
1. **Optimal Example Count**: 2-3 examples proved sufficient for consistent formatting
2. **Specificity Matters**: More specific prompts produced more reliable results
3. **Domain Knowledge**: The model performs best with general knowledge rather than specific local information
4. **Format Complexity**: Simpler formats showed more consistent results

## Recommendations
1. Keep formats simple and consistent
2. Provide clear, structured examples
3. Be cautious with numerical predictions
4. Include validation criteria for critical applications
5. Use multiple examples for complex classifications